<a href="https://colab.research.google.com/github/KanishkGar/Eva/blob/main/S5/Session_2_Step_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Code 1
Target: 
- Get the setup right so that we can start training. 
- Setup the transforms portion of the code. 
- No batchnorm/dropout, just a simple model.

Results: 
- Parameters: 17000
- Best Test Accuracy: 99%
- Best Training Accuracy: 99.5%

Analysis:
- It seems to be overfitting so we'll need to add layers to the model that will help make the train data simulate the test data more such as batchnorm and dropout
- There are too many parameters so a lighter model will be needed and the model will be changed


In [3]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [4]:
!/opt/bin/nvidia-smi
!nvcc --version

Sat Jan  2 17:21:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # input: 1x28x28
        # output: 8x14x14 
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 8, 3, padding=1),
            # RF - 3x3
            nn.ELU(),
            nn.Conv2d(8, 16, 3, padding=1),
            # RF - 5x5
            nn.ELU(),
            nn.Conv2d(16, 8, 1),
            nn.MaxPool2d(2, 2),
            # RF - 10x10
        )
        # input: 8x14x14
        # output: 16x7x7 
        self.conv2 = nn.Sequential(
            nn.Conv2d(8, 16, 3, padding=1),
            # RF - 12x12
            nn.ELU(),
            nn.Conv2d(16, 32, 3, padding=1),
            # RF - 14x14
            nn.ELU(),
            nn.Conv2d(32, 16, 1),
            nn.MaxPool2d(2, 2),
            # RF - 28x28
        )
        # input: 16x7x7
        # output: 32x1x1 
        self.conv3 = nn.Sequential(
            nn.Conv2d(16, 16, 3),
            # RF - 30x30
            nn.Conv2d(16, 16, 3),
            # RF - 32x32
            nn.ELU(),
            nn.Conv2d(16, 32, 3)
        )
      
                
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        
        x = x.view(x.size(0), -1)
        return F.log_softmax(x)

In [6]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
               ELU-2            [-1, 8, 28, 28]               0
            Conv2d-3           [-1, 16, 28, 28]           1,168
               ELU-4           [-1, 16, 28, 28]               0
            Conv2d-5            [-1, 8, 28, 28]             136
         MaxPool2d-6            [-1, 8, 14, 14]               0
            Conv2d-7           [-1, 16, 14, 14]           1,168
               ELU-8           [-1, 16, 14, 14]               0
            Conv2d-9           [-1, 32, 14, 14]           4,640
              ELU-10           [-1, 32, 14, 14]               0
           Conv2d-11           [-1, 16, 14, 14]             528
        MaxPool2d-12             [-1, 16, 7, 7]               0
           Conv2d-13             [-1, 16, 5, 5]           2,320
           Conv2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [7]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                        #add more transforms here later
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [14]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
        #tqdm._instances.clear()



def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [15]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.11549273878335953 batch_id=468 Accuracy=67.42: 100%|██████████| 469/469 [00:13<00:00, 35.70it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.1605, Accuracy: 9552/10000 (96%)



loss=0.13181056082248688 batch_id=468 Accuracy=96.43: 100%|██████████| 469/469 [00:13<00:00, 34.61it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0859, Accuracy: 9742/10000 (97%)



loss=0.0550151951611042 batch_id=468 Accuracy=97.64: 100%|██████████| 469/469 [00:13<00:00, 35.20it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0358, Accuracy: 9903/10000 (99%)



loss=0.001527292188256979 batch_id=468 Accuracy=99.50: 100%|██████████| 469/469 [00:13<00:00, 35.95it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0380, Accuracy: 9901/10000 (99%)



loss=0.042888905853033066 batch_id=468 Accuracy=99.55: 100%|██████████| 469/469 [00:13<00:00, 35.12it/s]



Test set: Average loss: 0.0360, Accuracy: 9894/10000 (99%)

